In [97]:
# Workspace problem with several narrow gaps

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
import os
import csv
from random import randint, random
import time

# (restrict tensorflow memory growth)
os.environ["CUDA_VISIBLE_DEVICES"]="1"
config = tf.ConfigProto()
config.gpu_options.allow_growth=True

In [99]:
# neural network parameters
mb_size = 256
h_Q_dim = 512
h_P_dim = 512

c = 0
# learning rate
lr = 1e-4

# problem dimensions
dim = 7
dataElements = dim*3 + 16 # sample (7D), init (7D), goal (7D), cond (16 points) //total = 37

z_dim = 3 # latent
X_dim = dim # samples
y_dim = dim # reconstruction of the original point
c_dim = dataElements - dim # dimension of conditioning variable

In [122]:
# define networks
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=[None, X_dim])
c = tf.placeholder(tf.float32, shape=[None, 30])
    
# Q
inputs_Q = tf.concat(axis=1, values=[X,c])

dense_Q1 = tf.layers.dense(inputs=inputs_Q, units=h_Q_dim, activation=tf.nn.relu)
dropout_Q1 = tf.layers.dropout(inputs=dense_Q1, rate=0.5)
dense_Q2 = tf.layers.dense(inputs=dropout_Q1, units=h_Q_dim, activation=tf.nn.relu)

z_mu = tf.layers.dense(inputs=dense_Q2, units=z_dim) # output here is z_mu
z_logvar = tf.layers.dense(inputs=dense_Q2, units=z_dim) # output here is z_logvar

# P
eps = tf.random_normal(shape=tf.shape(z_mu))
z = z_mu + tf.exp(z_logvar / 2) * eps
inputs_P = tf.concat(axis=1, values=[z,c])

dense_P1 = tf.layers.dense(inputs=inputs_P, units=h_P_dim, activation=tf.nn.relu)
dropout_P1 = tf.layers.dropout(inputs=dense_P1, rate=0.5)
dense_P2 = tf.layers.dense(inputs=dropout_P1, units=h_P_dim, activation=tf.nn.relu)

y = tf.layers.dense(inputs=dense_P2, units=X_dim) # fix to also output y

# training
########### comment in the one with 0 weight and uncomment the other ###########
w = [[1, 1, 1, 1, 1, 1, 1]];
# w = [[1, 1, 1, 0, 0, 0]];
recon_loss = tf.losses.mean_squared_error(labels=X, predictions=y, weights=w)

# TODO: fix loss function for angles going around
kl_loss = 10**-4 * 2 * tf.reduce_sum(tf.exp(z_logvar) + z_mu**2 - 1. - z_logvar, 1)

cvae_loss = tf.reduce_mean(kl_loss + recon_loss)

train_step = tf.train.AdamOptimizer(lr).minimize(cvae_loss)

sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())
it = 0;

(<tf.Tensor 'add:0' shape=(?, 3) dtype=float32>, <tf.Tensor 'Placeholder_1:0' shape=(?, 30) dtype=float32>)


In [123]:
saver = tf.train.Saver()
path_ = os.getcwd() + "/checkpoints_withoutV/model.ckpt"
print("path = ",path_)
print("numTrain = ",numTrain)
try:
    saver.restore(sess, path_)
    print("Model Restored!!")
except Exception as e:
    print("Could not restore checkpoint!")
    print(e)

('path = ', '/home/vernwalrahul/projects/LearningRoadmaps/cvae_learner/checkpoints_withoutV/model.ckpt')
('numTrain = ', 30551)
INFO:tensorflow:Restoring parameters from /home/vernwalrahul/projects/LearningRoadmaps/cvae_learner/checkpoints_withoutV/model.ckpt
Model Restored!!
(<tf.Tensor 'add:0' shape=(?, 3) dtype=float32>, <tf.Tensor 'Placeholder_1:0' shape=(?, 30) dtype=float32>)


In [124]:
# create test_conditions from data dir
def state_to_numpy(state):
        strlist = state.split()
        val_list = [float(s) for s in strlist]
        return np.array(val_list)
# x= []
# state_to_numpy(x)
def create_samples(G, directory):
    start = np.loadtxt(directory+"/start_node.txt")
    goal = np.loadtxt(directory+"/goal_node.txt")
    cond = np.loadtxt(directory+"/conditions.txt")
#     global state_to_numpy
    # cond = cond.split(",")
    path_nodes = []
    i = 0
    c_samples = []
    with open(directory + "/path_nodes.txt", 'r') as file:
        lines  = file.readlines()
        for line in lines:
            line = line.strip('\n')
#             print(line)
#             print("\n\n")
            if(not line == '-1'):
                s = state_to_numpy(G.node[str(int(start[i]))]['state'])
                g = state_to_numpy(G.node[str(int(goal[i]))]['state'])
                curr_node = np.array([])
                # print("Data = ",node_conf, s, g, cond)
#                 print("\n")
                curr_node = list(np.concatenate((s, g, cond)))
                c_samples.append(curr_node)
    return c_samples    

(<tf.Tensor 'add:0' shape=(?, 3) dtype=float32>, <tf.Tensor 'Placeholder_1:0' shape=(?, 30) dtype=float32>)


In [130]:
num_viz = 300
c_final = []

import networkx as nx
from random import choice
G = nx.read_graphml("graphs/herb_halton_1.graphml")
directory = "data/T1/9"
c_samples = create_samples(G, directory)

for i in range(num_viz):
    c_final.append(choice(c_samples))
c_final = np.float32(np.array(c_final))    
z_here = np.float32(np.random.randn(num_viz,z_dim))
print(c_final.dtype)
print(z_here.dtype)
print("c_final_dim = ", c_final.shape)
print("z_here.shape = ",z_here.shape)
# directly sample from the latent space (preferred, what we will use in the end)
# print(z, c)
y_viz, z_viz = sess.run([y, z], feed_dict={z: z_here, c: c_final})

print(y_viz.shape)
np.savetxt(directory+"/output_sample_node.txt", y_viz, delimiter=" ", fmt="%s")

float32
float32
('c_final_dim = ', (300, 30))
('z_here.shape = ', (300, 3))
(300, 7)
